# **재정정보 AI 검색 알고리즘 경진대회 - RAG, RAPTOR 적용**
> **RAG, RAPTOR 적용, Finetuning**

본 대회의 과제는 중앙정부 재정 정보에 대한 **검색 기능**을 개선하고 활용도를 높이는 질의응답 알고리즘을 개발하는 것입니다. <br>이를 통해 방대한 재정 데이터를 일반 국민과 전문가 모두가 쉽게 접근하고 활용할 수 있도록 하는 것이 목표입니다. <br><br>
베이스라인에서는 평가 데이터셋만을 활용하여 source pdf 마다 Vector DB를 구축한 뒤 langchain 라이브러리와 llama-2-ko-7b 모델을 사용하여 RAG 프로세스를 통해 추론하는 과정을 담고 있습니다. <br>( train_set을 활용한 훈련 과정은 포함하지 않으며, test_set  에 대한 추론만 진행합니다. )

---

💡 **NOTE**: 이 예제에서 사용한 모델 및 벡터 DB 

1. LLM : mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1
2. Embed Model : intfloat/multilingual-e5-large
3. Vector DB : FAISS

---

# 1. 라이브러리 설치하기

In [1]:
# 모델 가속화 및 메모리 관리
%pip install accelerate
%pip install -i https://pypi.org/simple/ bitsandbytes

# Transformer 기반 모델과 데이터셋 관련 라이브러리
%pip install transformers[torch] -U
%pip install datasets

# LangChain 관련 라이브러리
%pip install langchain
%pip install langchain_community
%pip install langchain-anthropic
%pip install langchain-huggingface


# 문서 처리 및 문서 임베딩 라이브러리
%pip install pymupdf4llm
%pip install sentence-transformers

# 벡터 데이터베이스 관련 라이브러리
%pip install faiss-cpu

# 데이터
%pip install umap-learn
%pip install scikit-learn
%pip install tiktoken

# env
%pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.5/417.5 kB 117.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.5/435.5 kB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 43.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple/
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 45.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 154.4 kB/s eta 0:00:00 0:00:01
   

# 2. 라이브러리 불러오기

In [4]:

import os
import re
import sys
import json
import unicodedata
from tqdm import tqdm
import torch
import numpy as np
import pandas as pd
import umap
import json
import pymupdf4llm
from typing import Dict, List, Optional, Tuple
import nest_asyncio

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
    BitsAndBytesConfig,
    Gemma2ForCausalLM,
    TextStreamer,
)
from accelerate import Accelerator
from sklearn.mixture import GaussianMixture

from langchain import hub
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.document_loaders import PDFPlumberLoader, PyMuPDFLoader, PyPDFLoader, UnstructuredPDFLoader
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.storage import LocalFileStore
from langchain.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever, MultiQueryRetriever
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.runnables import ConfigurableField
import logging
from dotenv import load_dotenv
import random

load_dotenv()
# 비동기 이벤트 루프를 재설정하거나 중첩할 수 있도록 허용
nest_asyncio.apply()

# 로깅 설정
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

# CUDA 메모리 할당이 필요할 때마다 메모리 블록을 확장
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# 시드 설정
seed = 42
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)


# 3. 초기 설정

In [14]:

class Config:
    def __init__(self, llm_name: str, embedding_name: str, concept: str) -> None:        
        # 모델별 설정 딕셔너리
        self.model_config = {
            "meta-llama/Meta-Llama-3.1-8B-Instruct": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "jjjguz/Llama-3.1-Korean-8B-Instruct-v1": {
                "quantization_config": None,
                "torch_dtype": "auto",
                "max_token": 256,
            },
            "rtzr/ko-gemma-2-9b-it": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "mindsignal/rtzr-ko-gemma-2-9b-it-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": "auto",
                "max_token": 450,
            },
            "mindsignal/upstage-SOLAR-10.7B-Instruct-v1.0-4bit-financesinfo-ver1": {
                "quantization_config": self.get_quantization_config(),
                "torch_dtype": torch.float16,
                "max_token": 512
            }
        }
        
        # Large Language Model
        self.llm_name = llm_name
        self.llm_config = self.model_config[self.llm_name]
        self.llm_obj = self.setup_llm()
        
        # 임베딩 모델 설정
        self.embedding_name = embedding_name
        self.embedding_obj = self.setup_embedding()
        
        # document
        self.chunk_size = 512
        self.chunk_overlap = 32
        self.bm25_w = 0.5
        self.faiss_w = 0.5
        
        # Data Path
        self.base_directory = "open/"
        self.test_csv_path = os.path.join(self.base_directory, "test.csv")
        
        # Output Path
        self.output_dir = "test_results"
        self.output_csv_file = f"{self.llm_name.split('/')[1]}_{concept}_submission.csv"
        os.makedirs(self.output_dir, exist_ok=True)
        
        # Langsmith 설정 (환경 변수)
        os.environ["LANGCHAIN_TRACING_V2"] = "true" # os.getenv("LANGCHAIN_TRACING_V2", "false")
        os.environ["LANGCHAIN_API_KEY"] = "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" # os.getenv("LANGCHAIN_API_KEY")
        os.environ["LANGCHAIN_PROJECT"] = "Docon-Financial-search" 
    
    def to_json(self):
        return json.dumps(self.__dict__)
    
    def get_quantization_config(self):
        """4-bit 양자화 설정을 반환하는 함수"""
        return BitsAndBytesConfig(
            load_in_4bit=True,  # 4-bit 양자화
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16  # 연산에 사용할 데이터 타입
        )
        
    def setup_embedding(self):
        """ 임베딩 모델 설정 """
        embed_id = self.embedding_name
        
        model_kwargs = {'device': 'cuda'}
        encode_kwargs = {'normalize_embeddings': True}
        embd = HuggingFaceEmbeddings(
            model_name=embed_id,
            model_kwargs=model_kwargs,
            encode_kwargs=encode_kwargs
        )
        
        store = LocalFileStore("./cache/")
        
        # Cache Embedding 사용
        cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
            underlying_embeddings=embd, 
            document_embedding_cache=store, 
            namespace=embed_id
        )
        
        return cached_embeddings
    
    def setup_llm(self):
        """LLM 설정 및 파이프라인 구성"""
        model_id = self.llm_name
        
        # 토크나이저 로드 및 설정
        tokenizer = AutoTokenizer.from_pretrained(model_id)
        tokenizer.use_default_system_prompt = False
        
        # 모델 로드 및 양자화 설정 적용
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="auto",
            quantization_config=self.llm_config["quantization_config"],
            trust_remote_code=True
        )
        
        # 모델을 여러 GPU에 할당
        accelerator = Accelerator()
        model = accelerator.prepare(model)
        
        print(f"#### [ model ] ####\n{model}\n###################")
        
        # 스트리머를 설정하여 토큰이 생성될 때마다 출력
        streamer = TextStreamer(tokenizer)

        # HuggingFacePipeline 객체 생성
        text_generation_pipeline = pipeline(
            model=model,
            tokenizer=tokenizer,
            task="text-generation",
            return_full_text=False,
            max_new_tokens=450,
            streamer=streamer,
        )

        hf = HuggingFacePipeline(pipeline=text_generation_pipeline)

        return hf

# 4. 문서 검색을 위한 RAPTOR 트리 구축

In [15]:

class RaptorClustering:
    def __init__(self, embed_obj, llm_obj):
        self.random_seed = 42
        self.embed_obj = embed_obj
        self.llm_obj = llm_obj
        logger.info(f'[RAPTOR] RaptorClustering 초기화')

    def global_cluster_embeddings(
        self, embeddings: np.ndarray, dim: int, n_neighbors: Optional[int] = None, metric: str = "cosine",
    ) -> np.ndarray:
        """
        임베딩 값에 대해 UMAP을 사용하여 전역 차원에서의 축소 수행
        차원을 'dim'으로 축소하여 클러스터링을 더 쉽게 만듬.
        """
        if n_neighbors is None:
            n_neighbors = int((len(embeddings) - 1) ** 0.5)
        logger.info(f"[RAPTOR] 글로벌 UMAP 차원 축소 수행 - dim={dim}, n_neighbors={n_neighbors}, metric={metric}")
        return umap.UMAP(
            n_neighbors=n_neighbors, n_components=dim, metric=metric
        ).fit_transform(embeddings)
    
    def local_cluster_embeddings(
        self, embeddings: np.ndarray, dim: int, num_neighbors: int = 10, metric: str = "cosine"
    ) -> np.ndarray:
        """
        임베딩에 대해 UMAP을 사용하여 지역 차원 축소를 수행. 
        일반적으로 전역 클러스터링 이후에 사용됨.
        """
        logger.info(f"[RAPTOR] 로컬 UMAP 차원 축소 수행 - dim={dim}, num_neighbors={num_neighbors}, metric={metric}")
        return umap.UMAP(
            n_neighbors=num_neighbors, n_components=dim, metric=metric
        ).fit_transform(embeddings)

    def get_optimal_clusters(
        self, embeddings: np.ndarray, max_clusters: int = 50
    ) -> int:
        """
        가우시안 혼합 모델(Gaussian Mixture Model)과 베이지안 정보 기준(BIC)을 사용하여 최적의 클러스터 수를 결정.
        """
        max_clusters = min(max_clusters, len(embeddings))  # 최대 클러스터 수와 임베딩의 길이 중 작은 값을 최대 클러스터 수로 설정
        n_clusters = np.arange(1, max_clusters)            # 1부터 최대 클러스터 수까지의 범위를 생성
        bics = []                                          # BIC 점수를 저장할 리스트
        
        logger.info("[RAPTOR] Gaussian Mixture Model(GMM)을 사용하여 최적의 클러스터 수 결정 중")
        for n in n_clusters: # 각 클러스터 수에 대해 반복
            gm = GaussianMixture(n_components=n, random_state=self.random_seed) # 가우시안 혼합 모델 초기화
            gm.fit(embeddings)              # 임베딩에 대해 모델 학습
            bic = gm.bic(embeddings)        # 학습된 bic 점수 추가
            bics.append(bic)
            logger.debug(f"[RAPTOR] {n}개의 클러스터에 대한 BIC: {bic}")
        
        optimal_clusters = n_clusters[np.argmin(bics)] # BIC 점수가 가장 낮은 클러스터 수를 반환
        logger.info(f"[RAPTOR] 결정된 최적의 클러스터 수: {optimal_clusters}")
        return optimal_clusters  

    def GMM_cluster(
        self, embeddings: np.ndarray, threshold: float
    ) -> Tuple[List[np.ndarray], int]:
        """
        임베딩을 GMM을 사용해 클러스터링합니다. 주어진 확률 임계값을 기준으로 클러스터링 실행
        """
        logger.info("[RAPTOR] GMM을 사용하여 임베딩 클러스터링 중")
        n_clusters = self.get_optimal_clusters(embeddings)  # 최적의 클러스터 수를 구합니다.
        gm = GaussianMixture(n_components=n_clusters, random_state=self.random_seed) # 가우시안 혼합 모델을 초기화합니다.
        gm.fit(embeddings)                                         # 임베딩에 대해 모델을 학습합니다.
        probs = gm.predict_proba(embeddings)                       # 임베딩이 각 클러스터에 속할 확률을 예측합니다.
        labels = [np.where(prob > threshold)[0] for prob in probs] # 임계값을 초과하는 확률을 가진 클러스터를 레이블로 선택합니다.
        logger.info(f"[RAPTOR] 임계값 {threshold}을 기준으로 클러스터 레이블 생성 완료")
        return labels, n_clusters                                  # 레이블과 클러스터 수(튜플)를 반환합니다.

    def perform_clustering(
        self, embeddings: np.ndarray, dim: int, threshold: float,
    ) -> List[np.ndarray]:
        """
        차원 축소 -> GMM을 사용한 클러스터링 -> 각 글로벌 클러스터 내에서의 로컬 클러스터링을 순서대로 수행
        """
        logger.info("[RAPTOR] 계층적 클러스터링 과정 시작")
        if len(embeddings) <= dim + 1:
            logger.warning("[RAPTOR] 데이터가 충분하지 않음 - 모든 데이터 포인트에 대해 단일 클러스터 반환")
            return [np.array([0]) for _ in range(len(embeddings))]

        # 글로벌 차원 축소
        reduced_embeddings_global = self.global_cluster_embeddings(embeddings, dim)
        # 글로벌 클러스터링
        global_clusters, n_global_clusters = self.GMM_cluster(
            reduced_embeddings_global, threshold
        )

        all_local_clusters = [np.array([]) for _ in range(len(embeddings))]
        total_clusters = 0

        # 각 글로벌 클러스터를 순회하며 로컬 클러스터링 수행
        for i in range(n_global_clusters):
            # 현재 글로벌 클러스터에 속하는 임베딩 추출
            global_cluster_embeddings_ = embeddings[
                np.array([i in gc for gc in global_clusters])
            ]

            if len(global_cluster_embeddings_) == 0:
                continue
            if len(global_cluster_embeddings_) <= dim + 1:
                # 작은 클러스터는 직접 할당으로 처리
                local_clusters = [np.array([0]) for _ in global_cluster_embeddings_]
                n_local_clusters = 1
            else:
                # 로컬 차원 축소 및 클러스터링
                reduced_embeddings_local = self.local_cluster_embeddings(
                    global_cluster_embeddings_, dim
                )
                local_clusters, n_local_clusters = self.GMM_cluster(
                    reduced_embeddings_local, threshold
                )

            # 로컬 클러스터 ID 할당, 이미 처리된 총 클러스터 수를 조정
            for j in range(n_local_clusters):
                local_cluster_embeddings_ = global_cluster_embeddings_[
                    np.array([j in lc for lc in local_clusters])
                ]
                indices = np.where(
                    (embeddings == local_cluster_embeddings_[:, None]).all(-1)
                )[1]
                for idx in indices:
                    all_local_clusters[idx] = np.append(
                        all_local_clusters[idx], j + total_clusters
                    )

            total_clusters += n_local_clusters
            logger.info(f"[RAPTOR] 글로벌 클러스터 {i} 처리 후 총 클러스터 수: {total_clusters}")

        return all_local_clusters


    def embed(self, texts: List[str]) -> np.ndarray:
        """
        텍스트 문서 목록에 대한 임베딩을 생성.
        embd 객체는 텍스트 목록을 받아 그 임베딩을 반환하는 `embed_documents` 메소드를 가지고 있음.
        """        
        text_embeddings = self.embed_obj.embed_documents(texts)  # 텍스트 문서들의 임베딩을 생성합니다.
        text_embeddings_np = np.array(text_embeddings)  # 임베딩을 numpy 배열로 변환합니다.
        return text_embeddings_np                       # 임베딩된 numpy 배열을 반환합니다.


    def embed_cluster_texts(self, texts: List[str]) -> pd.DataFrame:
        """
        텍스트 목록을 임베딩하고 클러스터링하여, 텍스트, 임베딩, 그리고 클러스터 라벨이 포함된 DataFrame을 반환합니다.
        이 함수는 임베딩 생성과 클러스터링을 단일 단계로 결합합니다.
        """
        logger.info("[RAPTOR] 텍스트 임베딩 및 클러스터링 시작")
        text_embeddings_np = self.embed(texts) # 임베딩 생성
        cluster_labels = self.perform_clustering(
            text_embeddings_np, 10, 0.1
        )  # 임베딩에 대해 클러스터링 수행
        df = pd.DataFrame()                   # 결과를 저장할 DataFrame 초기화
        df["text"] = texts                    # 원본 텍스트 저장
        df["embd"] = list(text_embeddings_np) # DataFrame에 리스트로 임베딩 저장
        df["cluster"] = cluster_labels        # 클러스터 라벨 저장
        logger.info("[RAPTOR] 클러스터링 완료 - 클러스터가 포함된 DataFrame 생성 완료")
        return df


    def fmt_txt(self, df: pd.DataFrame) -> str:
        """
        DataFrame에 있는 텍스트 문서를 단일 문자열로 포맷합니다.
        """
        logger.info("[RAPTOR] DataFrame의 텍스트를 단일 문자열로 포맷하는 중")
        unique_txt = df["text"].tolist()
        return "--- --- \n --- --- ".join(unique_txt)  # 텍스트 문서들을 특정 구분자로 결합하여 반환


    def embed_cluster_summarize_texts(
        self, texts: List[str], level: int
    ) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """
        텍스트 목록에 대해 임베딩, 클러스터링 및 요약을 수행합니다. 
        """
        # 텍스트를 임베딩하고 클러스터링하여 'text', 'embd', 'cluster' 열이 있는 데이터프레임을 생성합니다.
        logger.info(f"[RAPTOR] 텍스트 요약 시작 - 레벨 {level}")
        df_clusters = self.embed_cluster_texts(texts)
        expanded_list = []

        # 데이터프레임 항목을 문서-클러스터 쌍으로 확장하여 처리를 간단하게 합니다.
        for index, row in df_clusters.iterrows():
            for cluster in row["cluster"]:
                expanded_list.append(
                    {"text": row["text"], "embd": row["embd"], "cluster": cluster}
                )

            # 확장된 목록에서 새 데이터프레임을 생성합니다.
        expanded_df = pd.DataFrame(expanded_list)

        # 처리를 위해 고유한 클러스터 식별자를 검색합니다.
        all_clusters = expanded_df["cluster"].unique()

        logger.info(f"--Generated {len(all_clusters)} clusters--")

        # 요약
        template = """
        여기 중앙정부 재정 정보와 관련된 문서의 하위 집합이 있습니다.
        재정관련 제도, 지원과 운영방식 등에 대한 내용이 포함되어 있습니다.
        제공된 문서의 자세한 요약을 생성하세요.
        
        문서:
        {context}
        """
        prompt = ChatPromptTemplate.from_template(template)
        chain = prompt | self.llm_obj | StrOutputParser()

        # LLM을 사용하여 요약문을 생성합니다.       
        # 각 클러스터를 처리하고 요약을 생성합니다.
        summaries = []
        for i in all_clusters:
            df_cluster = expanded_df[expanded_df["cluster"] == i]
            formatted_txt = self.fmt_txt(df_cluster)
            summaries.append(chain.invoke({"context": formatted_txt}))
            
        # 요약, 해당 클러스터 및 레벨을 저장할 데이터프레임을 생성합니다.
        df_summary = pd.DataFrame(
            {
                "summaries": summaries,
                "level": [level] * len(summaries),
                "cluster": list(all_clusters),
            }
        )
        logger.info(f'클러스터링 및 요약 결과 : \ndf_clusters : \n{df_clusters}\n\n\ndf_summary\n{df_summary}')
        return df_clusters, df_summary

    def recursive_embed_cluster_summarize(
        self, texts: List[str], level: int = 1, n_levels: int = 3
    ) -> Dict[int, Tuple[pd.DataFrame, pd.DataFrame]]:
        """
        지정된 레벨까지 또는 고유 클러스터의 수가 1이 될 때까지 텍스트를 재귀적으로 임베딩, 클러스터링, 요약하고 각 레벨에서의 결과를 저장.
        """
        results = {}

        # 현재 레벨에 대해 임베딩, 클러스터링, 요약 수행
        logger.info(f"[RAPTOR] 레벨 {level}에서 임베딩, 클러스터링 및 요약 시작")
        df_clusters, df_summary = self.embed_cluster_summarize_texts(texts, level)

        # 현재 레벨의 결과 저장
        results[level] = (df_clusters, df_summary)

        # 추가 재귀가 가능하고 의미가 있는지 결정
        unique_clusters = df_summary["cluster"].nunique()
        if level < n_levels and unique_clusters > 1:
            logger.info(f"[RAPTOR] 재귀 조건 만족 - 다음 레벨 {level + 1}로 이동")
            new_texts = df_summary["summaries"].tolist()
            next_level_results = self.recursive_embed_cluster_summarize(
                new_texts, level + 1, n_levels
            )

            # 다음 레벨의 결과를 현재 결과 사전에 병합
            results.update(next_level_results)
        
        logger.info(f'results : \n{results}')
        return results

# 5. 데이터셋 전처리 및 벡터 DB 구축

In [12]:

class DataIngestionPipeline:
    def __init__(self, config:Config, raptor:RaptorClustering):
        self.config = config
        self.raptor = raptor
        logger.info("PDFProcessor 초기화 완료.")
                
    def _normalize_path(self, path):
        """ Path 유니코드 정규화 """
        normalized_path = unicodedata.normalize('NFD', path)
        logger.debug(f"정규화된 경로: {normalized_path}")
        return normalized_path
    
    def _preprocess_text(self, text: str) -> str:
        """ 텍스트 전처리 함수 """
        # 불필요한 공백 제거
        text = text.strip()
        # 마침표 뒤 및 "----" 전후의 줄바꿈을 제외한 모든 줄바꿈을 제거
        text = re.sub(r'(?<!\.)(?<!-----)(\n|\r\n)(?!-----)', ' ', text) # gemma
        # 공백 처리 및 불필요한 줄바꿈 제거
        text = re.sub(r'\s+', ' ', text)
        return text
    
    def _process_pdf(self, file_path) -> List[str]:
        """ PDF 파일 로드, 텍스트 추출 """
        logger.info(f"PDF 처리 중: {file_path}")
        docs_md_content = pymupdf4llm.to_markdown(file_path)
        
        # 전처리
        preprocess_content = self._preprocess_text(docs_md_content)
        
        # headers_to_split_on = [
        #     ("#", "Header 1"),
        #     ("##", "Header 2"),
        #     ("###", "Header 3"),
        # ]
        
        # md_header_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)
        # md_chunks = md_header_splitter.split_text(md_content)
        
        # 텍스트 데이터를 청크로 분할
        text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
            chunk_size=self.config.chunk_size, chunk_overlap=self.config.chunk_overlap
        )
        chunks = text_splitter.split_text(preprocess_content)
        logger.info(f"텍스트를 {len(chunks)}개의 청크로 분할 완료.")
        
        return chunks
    
    def init_retriever(self, df):
        """ PDF 데이터를 처리하고 벡터 데이터베이스 및 리트리버를 초기화 """
        pdf_databases = {}
        unique_paths = df['Source_path'].unique()

        for path in tqdm(unique_paths, desc="Processing PDFs"):
            # 경로 정규화 및 절대 경로 생성
            normalized_path = self._normalize_path(path)
            full_path = os.path.normpath(os.path.join(self.config.base_directory, normalized_path.lstrip('./'))) if not os.path.isabs(normalized_path) else normalized_path

            # pdf -> 벡터 DB 구축
            pdf_title = os.path.splitext(os.path.basename(full_path))[0]
            db_index_path = f"./{pdf_title}.faiss"
            
            
            if os.path.exists(db_index_path):
                logging.info(f"기존 벡터 DB 로드 중: {db_index_path}")
                vector_store = FAISS.load_local(db_index_path, embeddings=self.embed_obj, allow_dangerous_deserialization=True)
            else:
                print(f"PDF -> 벡터 DB 구축 중 : [{pdf_title}]...")
                
                # RAPTOR 트리 구축
                leaf_texts = self._process_pdf(full_path) # list[str]
                raptor_results = self.raptor.recursive_embed_cluster_summarize(
                    leaf_texts, level=1, n_levels=3
                )
                logger.info(f"RAPTOR 트리 구축 완료: {pdf_title}.")
            
                # 각 레벨의 요약을 추출하여 all_texts에 추가
                all_texts = leaf_texts.copy()
                for level in sorted(raptor_results.keys()):
                    summaries = raptor_results[level][1]["summaries"].tolist()  # 현재 레벨의 DataFrame에서 요약을 추출합니다.
                    all_texts.extend(summaries)                                 # 현재 레벨의 요약을 all_texts에 추가합니다.
                logger.info(f"{pdf_title}에 대해 요약 추가 완료.")
            
                # FAISS 벡터 데이터베이스 구축
                vector_store = FAISS.from_texts(texts=all_texts, embedding=self.config.embedding_obj)
                vector_store.save_local(db_index_path)
                logger.info(f"벡터 데이터베이스 저장 경로: {db_index_path}")
                
            raptor_retriever = vector_store.as_retriever(search_kwargs={"k": 3}).configurable_fields(
                search_kwargs = ConfigurableField(
                    id='search_kwargs_faiss',
                    name='Search Kwargs',
                    description='The search kwargs to use',
                )
            )
            
            pdf_databases[pdf_title] = {
                'db': vector_store,
                'retriever' : raptor_retriever,
            }
            logger.info(f"PDF 처리 완료: {pdf_title}.")
        
        logger.info("모든 PDF 처리 완료.")
        return pdf_databases
    

# 6. QA 생성 모듈

In [8]:
""" QA 프롬프트 템플릿 """

############ ChatPromptTemplate 사용 ###############3
# system_prompt_template = """ 
# 당신은 재정 정책 전문가입니다. 주어진 정보를 바탕으로, 명확하고 정확하게 답변해 해주세요.
# 답변을 생성할 때는 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 답변하세요.

# 아래 조건을 지켜서 답변을 생성해주세요 :
# 주어와 서술어를 사용하여 온전한 문장을 완성시키세요.
# 새로운 질문을 생성하지 마세요.
# 아래 정보의 내용을 참고해서 질문에 정확한 답변을 생성하세요.
# 기호나 공백을 제거하고 깔끔하게 답변하세요.

# 주어진 정보는 다음과 같습니다. : 
# {context}
# """
# financial_qa_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_prompt_template),
#         ("human", "{input}"),
#     ]
# )

############ PromptTemplate 사용 ###############3
template = """ 
당신은 재정 정책 전문가입니다. 주어진 정보를 바탕으로, 명확하고 정확하게 답변해 해주세요.
답변을 생성할 때는 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 답변하세요.

주어진 정보는 다음과 같습니다. : 
{context}

질문 :
{question}

새로운 질문을 생성하지 마세요.
주어진 정보의 내용을 참고해서 질문에 정확한 답변을 생성하세요.
기호나 공백을 제거하고 깔끔한 답변을 생성하세요.
주어와 서술어가 포함된 문장을 완성시켜서 답변을 생성하세요.
답변을 생성할 때 "답변:", "<eos>"와 같은 단어를 포함시키지 마세요.
"""
financial_qa_prompt = PromptTemplate.from_template(template) 


############ 다른 프롬프트 ###############3
# system_prompt_template = """
# 당신은 중앙정부 재정 정책 전문가입니다. 주어진 재정 정보를 바탕으로, 명확하고 정확한 답변을 제시하세요.
# 답변을 생성할 때에는 "Response:", "AI:" 등과 같은 불필요한 텍스트를 포함하지 말고, 핵심 정보만 명확하게 전달하세요.

# 주어진 질문에 대해 다음 단계에 따라 답변하세요:
# 1. 질문을 이해하고 필요한 재정 개념을 설명합니다.
# 2. 질문을 단계별로 분석하여 논리적인 답변을 구성합니다.
# 3. 관련 개념 간의 관계를 분석하여 맥락을 명확히 합니다.

# 답변은 간결하고 명확하게 작성하며, 주어진 질문에 대해 핵심만 파악하여 답변하세요.
# 단, 주어와 서술어를 적절히 사용하여 온전한 문장을 완성시켜 최대한 자연스럽게 답변해야 합니다.

# 다음 정보를 바탕으로 질문에 답하세요 :
# {context}            
# """

In [9]:

class FISearch:
    def __init__(self, config, prompts):
        self.config = config
        self.prompts = prompts
    
    def _normalize_string(self, s):
        """유니코드 정규화"""
        return unicodedata.normalize('NFC', s)
    
    def _format_docs(self, docs):
        # 문서의 페이지 내용을 이어붙여 반환합니다.
        return "\n\n".join(doc.page_content for doc in docs)
    
    def _clean_answer(self, answer):
        # 다양한 불필요한 텍스트 패턴을 제거
        patterns_to_remove = [
            r"^\s*###? Response:\s*",
            r"^\s*Response:\s*",
            r"^\s*AI:\s*",
            r"^\s*Quad:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*###? Response:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*Response:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*AI:\s*",
            r"^\s*\|\-\s*\n?\s*\n?\s*Quad:\s*",
            r"<eos>\s*$"
        ]
        for pattern in patterns_to_remove:
            answer = re.sub(pattern, "", answer)
        return answer.strip()
        
    def generate_answers(self, df, pdf_databases):
        """DataFrame의 각 질문에 대해 답변을 생성"""
        results = []
        
        # DataFrame의 각 행에 대해 처리
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Answering Questions"):
            # 소스 문자열 정규화
            source = self._normalize_string(row['Source'])
            question = row['Question']

            # 정규화된 키로 데이터베이스 검색
            normalized_keys = {self._normalize_string(k): v for k, v in pdf_databases.items()}
            raptor_retriever = normalized_keys[source]['retriever']
            
            # # RAG 체인 정의
            rag_chain = (
                {"context": raptor_retriever | self.format_docs, "question": RunnablePassthrough()}
                | self.prompts
                | self.config.llm_obj
                | StrOutputParser()
            )
            
            # # Retrieval Chain 생성
            # documents_chain = create_stuff_documents_chain(self.config.llm_obj, prompt=self.prompts)
            # rag_chain = create_retrieval_chain(ensemble_retriever, documents_chain) # EnsembleRetreiver + document chain
            
            
            
            # 답변 추론
            print(f"Question: {question}")
            # result = rag_chain.invoke({"input": query}) # Retreival Chain 사용
            result = rag_chain.invoke(question)
            print(f"답변: {result}\n================================================")
            # result = self._clean_answer(result)
            # print(f"후처리 답변: {result}\n================================================")
            
            # 결과 저장
            results.append({
                "Source": row['Source'],
                "Source_path": row['Source_path'],
                "Question": question,
                "Answer": result
            })

        return results

---

# 7. 최종 통합, 실행 및 저장

In [ ]:
if __name__ == "__main__":
    config = Config(
        llm_name="jjjguz/Llama-3.1-Korean-8B-Instruct-v1", 
        embedding_name="intfloat/multilingual-e5-large",
        concept="financial_analysis"
    )
    
    # RaptorClustering 객체 초기화
    raptor = RaptorClustering(
        embed_obj=config.embedding_obj,
        llm_obj=config.llm_obj
    )
    
    # 벡터 데이터베이스 및 리트리버 초기화
    df = pd.read_csv(config.test_csv_path)
    
    # 파이프라인 초기화
    dipipeline = DataIngestionPipeline(config=config, raptor=raptor)
    pdf_databases = dipipeline.init_retriever(df)
    
    # 검색 및 답변 생성
    search_engine = FISearch(config=config, prompts=financial_qa_prompt)
    results = search_engine.generate_answers(df, pdf_databases)
    
    # 제출 샘플 파일 로드 및 답변 추가
    submit_df = pd.read_csv(f"./open/sample_submission.csv")
    submit_df['Answer'] = [item['Answer'] for item in results]
    submit_df['Answer'] = submit_df['Answer'].fillna("데이콘")

    # 결과를 CSV 파일로 저장
    submit_df.to_csv(os.path.join(config.output_dir, config.output_csv_file), encoding='UTF-8-sig', index=False)
    
    print("결과가 성공적으로 저장되었습니다.")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

#### [ model ] ####
LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_lay

Processing PDFs:   0%|          | 0/9 [00:00<?, ?it/s]

PDF -> 벡터 DB 구축 중 : [중소벤처기업부_혁신창업사업화자금(융자)]...
INFO:__main__:PDF 처리 중: open/test_source/중소벤처기업부_혁신창업사업화자금(융자).pdf
INFO:__main__:텍스트를 10개의 청크로 분할 완료.
INFO:__main__:[RAPTOR] 레벨 1에서 임베딩, 클러스터링 및 요약 시작
INFO:__main__:[RAPTOR] 텍스트 요약 시작 - 레벨 1
INFO:__main__:[RAPTOR] 텍스트 임베딩 및 클러스터링 시작
INFO:__main__:[RAPTOR] 계층적 클러스터링 과정 시작
INFO:__main__:[RAPTOR] 클러스터링 완료 - 클러스터가 포함된 DataFrame 생성 완료
INFO:__main__:--Generated 1 clusters--
INFO:__main__:[RAPTOR] DataFrame의 텍스트를 단일 문자열로 포맷하는 중
<s> Human: 
        여기 중앙정부 재정 정보와 관련된 문서의 하위 집합이 있습니다.
        재정관련 제도, 지원과 운영방식 등에 대한 내용이 포함되어 있습니다.
        제공된 문서의 자세한 요약을 생성하세요.
        
        문서:
        ## 사 업 명 혁신창업사업화자금(융자) (5152-301) ## 1. 사업 코드 정보 |구분|기금|소관|실국(기관)|계정|분야|부문| |---|---|---|---|---|---|---| |코드|중소벤처기업 창업 및 진흥기금|중소벤처기업부|중소기업정책실||110|118| |명칭|||||산업·중소기업 및 에너지|창업 및 벤처| |구분|프로그램|단위사업|세부사업| |---|---|---|---| |코드|5100|5152|301| |명칭|창업환경조성|창업기업지원융자(기금)|혁신창업사업화자금(융자)| ## 구분 기금 소관 실국(기관) 계정 분야 부문--

Processing PDFs:  11%|█         | 1/9 [01:01<08:09, 61.19s/it]

...

PDF -> 벡터 DB 구축 중 : [국토교통부_행복주택출자]...
INFO:__main__:PDF 처리 중: open/test_source/국토교통부_행복주택출자.pdf
INFO:__main__:텍스트를 12개의 청크로 분할 완료.
INFO:__main__:[RAPTOR] 레벨 1에서 임베딩, 클러스터링 및 요약 시작
INFO:__main__:[RAPTOR] 텍스트 요약 시작 - 레벨 1
INFO:__main__:[RAPTOR] 텍스트 임베딩 및 클러스터링 시작
INFO:__main__:[RAPTOR] 계층적 클러스터링 과정 시작
INFO:__main__:[RAPTOR] 글로벌 UMAP 차원 축소 수행 - dim=10, n_neighbors=3, metric=cosine
INFO:__main__:[RAPTOR] GMM을 사용하여 임베딩 클러스터링 중
INFO:__main__:[RAPTOR] Gaussian Mixture Model(GMM)을 사용하여 최적의 클러스터 수 결정 중
INFO:__main__:[RAPTOR] 결정된 최적의 클러스터 수: 3
INFO:__main__:[RAPTOR] 임계값 0.1을 기준으로 클러스터 레이블 생성 완료
INFO:__main__:[RAPTOR] 글로벌 클러스터 0 처리 후 총 클러스터 수: 1
INFO:__main__:[RAPTOR] 글로벌 클러스터 1 처리 후 총 클러스터 수: 2
INFO:__main__:[RAPTOR] 글로벌 클러스터 2 처리 후 총 클러스터 수: 3
INFO:__main__:[RAPTOR] 클러스터링 완료 - 클러스터가 포함된 DataFrame 생성 완료
INFO:__main__:--Generated 3 clusters--
INFO:__main__:[RAPTOR] DataFrame의 텍스트를 단일 문자열로 포맷하는 중
<s> Human: 
        여기 중앙정부 재정 정보와 관련된 문서의 하위 집합이 있습니다.
        재정관련

Processing PDFs:  56%|█████▌    | 5/9 [53:08<51:57, 779.37s/it]  

PDF -> 벡터 DB 구축 중 : [「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》]...
INFO:__main__:PDF 처리 중: open/test_source/「FIS 이슈 & 포커스」 22-4호 《중앙-지방 간 재정조정제도》.pdf
INFO:__main__:텍스트를 82개의 청크로 분할 완료.
INFO:__main__:[RAPTOR] 레벨 1에서 임베딩, 클러스터링 및 요약 시작
INFO:__main__:[RAPTOR] 텍스트 요약 시작 - 레벨 1
INFO:__main__:[RAPTOR] 텍스트 임베딩 및 클러스터링 시작
INFO:__main__:[RAPTOR] 계층적 클러스터링 과정 시작
INFO:__main__:[RAPTOR] 글로벌 UMAP 차원 축소 수행 - dim=10, n_neighbors=9, metric=cosine
INFO:__main__:[RAPTOR] GMM을 사용하여 임베딩 클러스터링 중
INFO:__main__:[RAPTOR] Gaussian Mixture Model(GMM)을 사용하여 최적의 클러스터 수 결정 중
INFO:__main__:[RAPTOR] 결정된 최적의 클러스터 수: 12
INFO:__main__:[RAPTOR] 임계값 0.1을 기준으로 클러스터 레이블 생성 완료
INFO:__main__:[RAPTOR] 글로벌 클러스터 0 처리 후 총 클러스터 수: 1
INFO:__main__:[RAPTOR] 글로벌 클러스터 1 처리 후 총 클러스터 수: 2
INFO:__main__:[RAPTOR] 글로벌 클러스터 2 처리 후 총 클러스터 수: 3
INFO:__main__:[RAPTOR] 글로벌 클러스터 3 처리 후 총 클러스터 수: 4
INFO:__main__:[RAPTOR] 글로벌 클러스터 4 처리 후 총 클러스터 수: 5
INFO:__main__:[RAPTOR] 글로벌 클러스터 5 처리 후 총 클러스터 수: 6
INFO:

This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.



























INFO:__main__:[RAPTOR] DataFrame의 텍스트를 단일 문자열로 포맷하는 중
<s> Human: 
        여기 중앙정부 재정 정보와 관련된 문서의 하위 집합이 있습니다.
        재정관련 제도, 지원과 운영방식 등에 대한 내용이 포함되어 있습니다.
        제공된 문서의 자세한 요약을 생성하세요.
        
        문서:
        ||• 민간보조|16.8|19.1|21.1|23.1|22.9|8.2%| |정부총지출(조원)||428.8|469.6|512.3|558.0|607.7|9.1%| |정부총지출 대비 보조금 비율||15.6%|16.6%|16.9%|17.5%|16.8%|1.9%| **주 1) 국회예산 확정 기준임.** 2) e-나라도움 국고보조금통합관리시스템에 기반함. **자료:기획재정부(2022.9.), 「2022년 국고보조사업 연장평가 보고서(Ⅰ)」, p.13.를 바탕으로 기획재정부의 열린재정을 일부 참고하여 작성함(www.openfiscaldata.go.kr,** 검색일: 2022.10.25. 기준). (분야 및 부문별 현황) 2022년 기준 사회복지--- --- 
 --- --- 현황) 2022년 기준 사회복지 분야가 약 55.7조원으로 전체 국고보조금 중 절반 이상인 54.4%의 비중을 차지 - 농림수산 분야가 약 10.9조원(10.6%), 환경 분야가 7.9조원(7.7%), 산업·중소기업 분야가 7.6조원(7.4%), 문화 및 관광 분야가 5.5조원(5.4%) 순으로 규모가 큼 - 저출산·고령화의 심화와 사회복지 수요의 증가로 인하여 기초연금, 영·유아보육료, 가족양육수당 등 사회 복지 분야 지출 증가에 기인[9)] <표 8> 2022년 분야별 국고보조금 비중--- --- 
 --- --- 분야별 국고보조금 비중 |분야|사회복지|농림수산|환경|산업·중소기업 및 에너지|문화 및 관광|교통 및 물류|보건|국토 및 지역개발| |---|---|---|

---